In [7]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv("FOI 1760 2021/FOI 1760-2021.csv", low_memory = False)

In [3]:
df.head()

,Travel Location Name,Travel Location No,Transport Mode,Travel Date,Transaction Type,Count of Taps
0,Abbey Road DLR,750,Rail,01/01/2020 00:00,Exit,224
1,Abbey Road DLR,750,Rail,01/01/2020 00:00,PVal,230
2,Abbey Road DLR,750,Rail,01/01/2020 00:00,Entry,229
3,Abbey Road DLR,750,Rail,02/01/2020 00:00,Exit,406
4,Abbey Road DLR,750,Rail,02/01/2020 00:00,PVal,552


In [4]:
df = df[df['Transport Mode'] == "Rail" ]

In [5]:
df.rename(columns={'Count of Taps':'Amount'}, inplace=True)
df["Travel Date"] = pd.to_datetime(df["Travel Date"])
dfs = df.groupby(["Travel Date", "Travel Location Name"]).Amount.sum().reset_index()
# dfs = dfs.groupby(dfs.index.week).cumsum().reset_index()
dfs

,Travel Date,Travel Location Name,Amount
0,2020-01-01,Abbey Road DLR,683
1,2020-01-01,Abbey Wood,3035
2,2020-01-01,Acton Central,1962
3,2020-01-01,Acton Main Line,242
4,2020-01-01,Acton Town,7355
...,...,...,...
218136,2020-12-11,Woodside Park,2747
218137,2020-12-11,Woolwich Arsenal DLR,13720
218138,2020-12-11,Woolwich Arsenal NR,5271
218139,2020-12-11,Woolwich Dockyard,617


In [8]:
dfs['Amount']=np.round(dfs['Amount']/1000)

In [65]:
dfs

,Travel Date,Travel Location Name,Amount
0,2020-01-01,Abbey Road DLR,1.0
1,2020-01-01,Abbey Wood,3.0
2,2020-01-01,Acton Central,2.0
3,2020-01-01,Acton Main Line,0.0
4,2020-01-01,Acton Town,7.0
...,...,...,...
218136,2020-12-11,Woodside Park,3.0
218137,2020-12-11,Woolwich Arsenal DLR,14.0
218138,2020-12-11,Woolwich Arsenal NR,5.0
218139,2020-12-11,Woolwich Dockyard,1.0


In [9]:
# first lockdown 23 mar, week after
dfs["Travel Date"] = pd.to_datetime(dfs["Travel Date"])
before_ld1 = dfs[(dfs["Travel Date"] >= '2020-03-23 00:00:00') & (dfs["Travel Date"] < '2020-03-30 00:00:00')]
before_ld1 = before_ld1.groupby(["Travel Location Name"]).Amount.mean().reset_index()

In [10]:
# import pprint

with open('tfl_stations.json') as f:
  stations = json.load(f)

# print(json.dumps(stations, allow_nan = True,
#                          indent = 6))
# pprint.pprint(stations)

In [11]:
stations_df = pd.json_normalize(stations["features"])

In [12]:
stations_df.head()

,type,properties.id,properties.name,properties.nlc_id,properties.lines,properties.cartography.labelX,properties.alt_id,properties.zone,geometry.type,geometry.coordinates,properties.cartography.labelY,properties.cartography.display_name,properties.altmodeid,properties.altmodeid2
0,Feature,940GZZLUACT,Acton Town,0500,"[{'name': 'District'}, {'name': 'Piccadilly', ...",30,1,3,Point,"[-0.279916688851386, 51.50264359676248]",NaN,NaN,NaN,NaN
1,Feature,940GZZLUACY,Archway,0508,"[{'name': 'Northern', 'nightopened': 2016}]",-30,8,2/3,Point,"[-0.134745288767581, 51.565370996797775]",NaN,NaN,NaN,NaN
2,Feature,940GZZLUADE,Aldgate East,0503,"[{'name': 'District'}, {'name': 'Hammersmith &...",30,3,1,Point,"[-0.071876588767481, 51.51514719676551]",NaN,NaN,NaN,NaN
3,Feature,940GZZLUAGL,Angel,0507,"[{'name': 'Northern'}, {'name': 'Crossrail 2',...",1,7,1,Point,"[-0.10612998877245, 51.53182149677656]",-35.0,NaN,NaN,NaN
4,Feature,940GZZLUALD,Aldgate,0502,"[{'name': 'Circle'}, {'name': 'Metropolitan'}]",25,2,1,Point,"[-0.075715588769394, 51.51409639676498]",20.0,NaN,NaN,NaN


In [13]:
before_ld1.rename(columns={'Travel Location Name':'properties.name'}, inplace=True)

In [14]:
before_ld1.head()

,properties.name,Amount
0,Abbey Road DLR,0.142857
1,Abbey Wood,2.000000
2,Acton Central,0.857143
3,Acton Main Line,0.000000
4,Acton Town,2.285714


In [15]:
df_final = pd.merge(before_ld1, stations_df, how='right',on = 'properties.name')
df_final.head()

,properties.name,Amount,type,properties.id,properties.nlc_id,properties.lines,properties.cartography.labelX,properties.alt_id,properties.zone,geometry.type,geometry.coordinates,properties.cartography.labelY,properties.cartography.display_name,properties.altmodeid,properties.altmodeid2
0,Acton Town,2.285714,Feature,940GZZLUACT,0500,"[{'name': 'District'}, {'name': 'Piccadilly', ...",30,1,3,Point,"[-0.279916688851386, 51.50264359676248]",NaN,NaN,NaN,NaN
1,Archway,2.571429,Feature,940GZZLUACY,0508,"[{'name': 'Northern', 'nightopened': 2016}]",-30,8,2/3,Point,"[-0.134745288767581, 51.565370996797775]",NaN,NaN,NaN,NaN
2,Aldgate East,2.000000,Feature,940GZZLUADE,0503,"[{'name': 'District'}, {'name': 'Hammersmith &...",30,3,1,Point,"[-0.071876588767481, 51.51514719676551]",NaN,NaN,NaN,NaN
3,Angel,2.285714,Feature,940GZZLUAGL,0507,"[{'name': 'Northern'}, {'name': 'Crossrail 2',...",1,7,1,Point,"[-0.10612998877245, 51.53182149677656]",-35.0,NaN,NaN,NaN
4,Aldgate,1.142857,Feature,940GZZLUALD,0502,"[{'name': 'Circle'}, {'name': 'Metropolitan'}]",25,2,1,Point,"[-0.075715588769394, 51.51409639676498]",20.0,NaN,NaN,NaN


In [16]:
df_final.rename(columns={'Amount':'properties.footfall'}, inplace=True)

In [17]:
df_final = df_final[['type', 'properties.id', 'properties.name', 'properties.nlc_id', 'properties.lines', 
                     'properties.cartography.labelX', 'properties.cartography.labelY', 'properties.cartography.display_name',
                     'properties.altmodeid', 'properties.altmodeid2', 'properties.alt_id', 'properties.zone', 'properties.footfall', 
                     'geometry.type', 'geometry.coordinates'
                    ]]
# {'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'},
#          'type': 'name'},
#  'features': [{'geometry': {'coordinates': [-0.279916688851386,
#                                             51.50264359676248],
#                             'type': 'Point'},
#                'properties': {'alt_id': 1,
#                               'cartography': {'labelX': 30},
#                               'id': '940GZZLUACT',
#                               'lines': [{'name': 'District'},
#                                         {'name': 'Piccadilly',
#                                          'nightopened': 2016}],
#                               'name': 'Acton Town',
#                               'nlc_id': '0500',
#                               'zone': '3'},
#                'type': 'Feature'},

In [18]:
df_final.head()

,type,properties.id,properties.name,properties.nlc_id,properties.lines,properties.cartography.labelX,properties.cartography.labelY,properties.cartography.display_name,properties.altmodeid,properties.altmodeid2,properties.alt_id,properties.zone,properties.footfall,geometry.type,geometry.coordinates
0,Feature,940GZZLUACT,Acton Town,0500,"[{'name': 'District'}, {'name': 'Piccadilly', ...",30,NaN,NaN,NaN,NaN,1,3,2.285714,Point,"[-0.279916688851386, 51.50264359676248]"
1,Feature,940GZZLUACY,Archway,0508,"[{'name': 'Northern', 'nightopened': 2016}]",-30,NaN,NaN,NaN,NaN,8,2/3,2.571429,Point,"[-0.134745288767581, 51.565370996797775]"
2,Feature,940GZZLUADE,Aldgate East,0503,"[{'name': 'District'}, {'name': 'Hammersmith &...",30,NaN,NaN,NaN,NaN,3,1,2.000000,Point,"[-0.071876588767481, 51.51514719676551]"
3,Feature,940GZZLUAGL,Angel,0507,"[{'name': 'Northern'}, {'name': 'Crossrail 2',...",1,-35.0,NaN,NaN,NaN,7,1,2.285714,Point,"[-0.10612998877245, 51.53182149677656]"
4,Feature,940GZZLUALD,Aldgate,0502,"[{'name': 'Circle'}, {'name': 'Metropolitan'}]",25,20.0,NaN,NaN,NaN,2,1,1.142857,Point,"[-0.075715588769394, 51.51409639676498]"


In [19]:
import numpy as np
df_final = df_final.replace(np.nan, '') # All data frame



In [20]:
def df_to_formatted_json(df, sep="."):
    """
    The opposite of json_normalize
    """
    result = []
    for idx, row in df.iterrows():
        parsed_row = {}
        for col_label,v in row.items():
            keys = col_label.split(".")

            current = parsed_row
            for i, k in enumerate(keys):
                if i==len(keys)-1:
                    current[k] = v
                else:
                    if k not in current.keys():
                        current[k] = {}
                    current = current[k]
        # save
        result.append(parsed_row)
    return result

In [21]:
# df_feature1[0]['properties']['altmodeid']

In [22]:
df_feature= df_to_formatted_json(df_final)


In [23]:
df_dict= {"type": "FeatureCollection",
#           "crs": {
#               "type": "name",
#               "properties": {
#                   "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
#             }
#           }
         }
df_dict['features'] = df_feature
# df_json = json.dumps(df_dict)
# df_json = json.loads(df_dict)

with open('after_ld1.json', 'w') as fp:
    json.dump(df_dict, fp)
#     simplejson.dumps(df_dict, fp,ignore_nan=True)

In [81]:
import numpy as np
# df_feature1 = df_feature.replace(np.nan, '') # All data frame


df_feature1=['' if x is np.nan else x for x in df_feature]

In [84]:
data = pd.read_csv("applemobilitytrends-2021-03-22_london.csv", encoding="ISO-8859-1", low_memory=False)

In [85]:
data=data.loc[data['region'] == "London"].transpose().iloc[2:]

In [86]:
data = data.rename(columns=data.iloc[0])
data=data.iloc[4:]

In [87]:
data.head()

,driving,transit,walking
2020/1/13,100,100,100
2020/1/14,107.65,105.14,108.89
2020/1/15,111.2,107.06,116.84
2020/1/16,108.29,106.08,118.82
2020/1/17,115.08,115.13,132.18


In [88]:
data['date'] = data.index
data = data.reset_index()
data = data.drop("index", axis = 1)

In [89]:
# from dateutil import parser
# date_col = []
# for i in data["date"]:
#   date_col.append(parser.parse(data.date[i])) 
# # data["datetime"][i]


In [90]:
x = data.index
y = data.driving
df = pd.DataFrame(data=y,index=x)


In [91]:
import numpy as np
test_pecent = 0.16667 
len(df)*test_pecent
test_point = np.round(len(df)*test_pecent) 
test_index = int(len(df) - test_point) 
train = df.iloc[:test_index]
test = df.iloc[test_index:]

plt.figure(figsize=(12,8))
plt.xlim(0, 60)

print(len(df)) # 601
print(len(train))
print(len(test))
#train.plot()
#test.plot()
plt.plot(train)
plt.plot(test)

NameError: name 'plt' is not defined